La base essentielle a connaitre : 

# Reindexation

UNE METHODE IMPORTANTE POUR LES OBJET PANDAS EST REINDEX, qui consiste à créer un nouvel objet dont les données se conforment à un nouvel index. Prenons exemple:

In [2]:
import pandas as pd

obj = pd.Series([4.5, 7.2, -5.3, 3.6], index=['d', 'b', 'a', 'c'])
obj

d    4.5
b    7.2
a   -5.3
c    3.6
dtype: float64

l'appel à reindex sur cette série réorganise les données en fonction du nouvel index, en introduisant des valeurs manquantes si aucune valeur d'index n'etait déja présente:

In [3]:
obj2 = obj.reindex(['a', 'b', 'c', 'd', 'e'])
obj2

a   -5.3
b    7.2
c    3.6
d    4.5
e    NaN
dtype: float64

Pour les données ordonnées, comme les séries chronologique, il peut etre souhaitable de faire une certaine interpolation ou de remplir des valeur lors de la réindexation, l'option METHODE nous permet de le faire, en spécifiant une méthode telle que ffill, qui préremplit les valeur :

In [4]:
obj3 = pd.Series(['bleu', 'pourpre', 'jaune'], index=[0, 2, 4])
obj3

0       bleu
2    pourpre
4      jaune
dtype: object

In [5]:
obj3.reindex(range(6), method='ffill')

0       bleu
1       bleu
2    pourpre
3    pourpre
4      jaune
5      jaune
dtype: object

Avec un dataframe, reindex peut modifier soit l'index (ligne), soit les colonnes, ou les deux. Si seule une séquence est passé, elle reindexe les lignes du résultat : 

In [8]:
import numpy as np
frame = pd.DataFrame(np.arange(9).reshape((3, 3)), index=['a', 'c', 'd'], columns=['Ohio', 'Texas', 'Californie'])
frame

,Ohio,Texas,Californie
a,0,1,2
c,3,4,5
d,6,7,8
